In [2]:
search_list = [
    {'site_id':'MLB', 'results':[{'id': 'MLB1266303120', 'site_id': 'MLB', 'title': 'Body Longo+body Curt... C/20peças'}]},
    {'site_id':'MLB', 'results':[{'id': 'MLB1266303121', 'site_id': 'MLB', 'title': 'Body Curto'}]}
]
items = [search['results'] for search in search_list]
items

[[{'id': 'MLB1266303120',
   'site_id': 'MLB',
   'title': 'Body Longo+body Curt... C/20peças'}],
 [{'id': 'MLB1266303121', 'site_id': 'MLB', 'title': 'Body Curto'}]]

In [ ]:
# keyValList = ['Bebês', 'Beleza e Cuidado Pessoal', 'Esportes e Fitness', 'Saúde']
#     base_categories = list(
#         filter(lambda d: d['name'] in ['Bebês'], base_categories))

In [ ]:
# leaf_categories = []
    # for base_category in tqdm(base_categories, desc="Crawling categories: "):
    #     category = api.get_category(base_category['id'])
    #     api.get_leaf_categories(category, leaf_categories)
    # formated_leaf_categories = format_categories(leaf_categories, TODAY)
    # db.insert_bulk_categories(formated_leaf_categories)
    # logger.info('The leaf_categories list contains %s element(s)',
    #             len(leaf_categories))

In [ ]:
# for leaf_category in tqdm(leaf_categories, desc="Items: "):
    #     logger.info('Processing leaf_category %s', leaf_category["name"])

    #     category_search = api.search_items(
    #         SITE_ID, {'category': leaf_category['id']})
    #     available_filters = category_search['available_filters']
    #     available_sorts = category_search['available_sorts']
    #     total_items = category_search['paging']['total']
    #     limit = category_search['paging']['limit']
    #     category_id = category_search['filters'][0]['values'][0]['id']

    #     logger.info('The leaf_category %s has %s item(s)',
    #                 leaf_category["name"], total_items)

    #     if total_items <= API_REQUEST_QUOTA:
    #         params = {'category': category_id}
    #         items = api.get_items(
    #             SITE_ID, params, total_items, limit, API_REQUEST_QUOTA)
    #         formated_items = format_items(items, TODAY)
    #         db.insert_bulk_items(formated_items)

    #         logger.info('Able to collect %s of %s item(s) for leaf_category %s',
    #                     len(items), total_items, leaf_category['name'])
    #     else:
    #         optimized_filters = optimize_filters(
    #             available_filters, total_items)
    #         filters_combinations = get_filter_combinations(
    #             optimized_filters, available_sorts)

    #         logger.info('Generating %s filters to be able to colect the data.',
    #                     len(filters_combinations))

    #         for filters_combination in tqdm(filters_combinations, desc='Fiters: '):

    #             params = {**{'category': category_id}, **filters_combination}
    #             cs = api.search_items(SITE_ID, params)
    #             total_items = cs['paging']['total']
    #             limit = cs['paging']['limit']

    #             items = api.get_items(
    #                 SITE_ID, params, total_items, limit, API_REQUEST_QUOTA)
    #             formated_items = format_items(items, TODAY)
    #             db.insert_bulk_items(formated_items)

    #             logger.info('Able to collect %s of %s item(s) for leaf_category %s and filter %s',
    #                         len(items), total_items, leaf_category['name'], filters_combination)

    #             numb_distinct_items = db.count_disctinct_items(
    #                 SITE_ID, category_id[3:], TODAY)

    #             if numb_distinct_items >= total_items * DISTINCT_ITEMS_THRESHOLD:
    #                 break

In [1]:
import psycopg2
from psycopg2 import Error

In [ ]:
pg_host="127.0.0.53"
pg_database="MercadoLivre" 
pg_user="postgres"
pg_password="changeme"

In [1]:
def readData(site_id, category_id, last_run):
    try:
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)

        cur = conn.cursor()
        postgres_read_query = "SELECT * FROM base_categories WHERE site_id = %s AND category_id = %s AND last_run = %s;"
        cur.execute(postgres_read_query, (site_id, category_id, last_run))

        rows = cur.fetchall()
        count = cur.rowcount
        print(count, "Records were read successfully from 'base_categories' table")
        return rows
    except (Exception, psycopg2.Error) as error:
        print("Failed to read from 'base_categories' table", error)
    finally:
        if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed")

def readAllData():
    try:
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)

        cur = conn.cursor()
        postgres_read_query = "SELECT * FROM base_categories;"
        cur.execute(postgres_read_query)

        rows = cur.fetchall()
        count = cur.rowcount
        print(count, "Records were read successfully from 'base_categories' table")
        return rows
    except (Exception, psycopg2.Error) as error:
        print("Failed to read from 'base_categories' table", error)
    finally:
        if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed")

def insertData(site_id, category_id, last_run, category_json):
    try:
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)

        cur = conn.cursor()

        postgres_insert_query = "INSERT INTO base_categories (site_id, category, last_run, category_json) VALUES (%s,%s,%s,%s)"
        record_to_insert = (site_id, category_id, last_run, category_json)
        cur.execute(postgres_insert_query, record_to_insert)

        conn.commit()
        count = cur.rowcount
        print(count, "Record inserted successfully into 'base_categories' table")
        return record_to_insert
    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into 'base_categories' table", error)
    finally:
        if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed")

def updateData(site_id, category_id, last_run, category_json):
    try:
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)

        cur = conn.cursor()

        print("Table Before updating record ")
        sql_select_query = """select * from base_categories where site_id = %s AND category_id = %s AND last_run = %s"""
        cur.execute(sql_select_query, (site_id, category_id, last_run))
        record = cur.fetchone()
        print(record)

        # Update single record now
        sql_update_query = """Update base_categories set category_json = %s where site_id = %s AND category_id = %s AND last_run = %s"""
        cur.execute(sql_update_query, (category_json, site_id, category_id, last_run))
        conn.commit()
        count = cur.rowcount
        print(count, "Record Updated successfully ")

        print("Table After updating record ")
        sql_select_query = """select * from base_categories where site_id = %s AND category_id = %s AND last_run = %s"""
        cur.execute(sql_select_query, (site_id, category_id, last_run,))
        record = cur.fetchone()
        print(record)
        return record
    except (Exception, psycopg2.Error) as error:
        print("Error in update operation", error)
    finally:
        if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed")

def deleteData(site_id, category_id, last_run):
    try:
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)

        cur = conn.cursor()

        sql_delete_query = """Delete from base_categories where site_id = %s AND category_id = %s AND last_run = %s"""
        cur.execute(sql_delete_query, (site_id, category_id, last_run,))
        conn.commit()
        count = cur.rowcount
        print(count, "Record deleted successfully ")

    except (Exception, psycopg2.Error) as error:
        print("Error in Delete operation", error)
    finally:
        if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed")
    
def bulkInsert(records):
    try:
        connection = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)
        cursor = connection.cursor()
        sql_insert_query = """ INSERT INTO base_categories (site_id, category_id, last_run, category_json) 
                           VALUES (%s,%s,%s,%s) """

        # executemany() to insert multiple rows
        result = cursor.executemany(sql_insert_query, records)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into 'base_categories' table")

    except (Exception, psycopg2.Error) as error:
        print("Failed inserting record into 'base_categories' table {}".format(error))
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

records_to_insert = [(4, 'LG', 800), (5, 'One Plus 6', 950)]
# bulkInsert(records_to_insert)

def updateInBulk(records):
    try:
        ps_connection = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)
        cursor = ps_connection.cursor()

        # Update multiple records
        sql_update_query = """Update base_categories set category_json = %s where site_id = %s AND category_id = %s AND last_run = %s"""
        cursor.executemany(sql_update_query, records)
        ps_connection.commit()

        row_count = cursor.rowcount
        print(row_count, "Records Updated")

    except (Exception, psycopg2.Error) as error:
        print("Error while updating PostgreSQL table", error)
    finally:
        if ps_connection:
            cursor.close()
            ps_connection.close()
            print("PostgreSQL connection is closed")

tuples = [(750, 4), (950, 5)]
# updateInBulk(tuples)

def deleteInBulk(records):
    try:
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)
        cur = conn.cursor()
        ps_delete_query = """Delete from base_categories where site_id = %s AND category_id = %s AND last_run = %s"""
        cur.executemany(ps_delete_query, records)
        conn.commit()

        row_count = cur.rowcount
        print(row_count, "Record Deleted")

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

    finally:
        if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed")